In [1]:


import os
import re
from scipy.io import loadmat
import mat73

from collections import Counter
import pandas as pd
import numpy as np
import mne

import seaborn as sns
import matplotlib.pyplot as plt

from utils import *


In [2]:
TIMELINE_F = '/mnt/ernie_main/000_Data/SEEG/SEEG_Periictal/data/Extracted_Per_Event_Interictal/all_time_data_01042023_212306.csv'
SEEG_FOLDER = '/mnt/ernie_main/000_Data/SEEG/SEEG_Entire_EMU_Downloads/data/'

In [3]:
def load_mat(f):
    try:
        return loadmat(f)
    except NotImplementedError:
        return mat73.loadmat(f)

seeg_timeline_df = pd.read_csv(TIMELINE_F, delimiter="\t")
seeg_timeline_df.head(5)

,Type,Pat ID,Event Number (Matches EMU Final Report),Date (MM:DD:YYYY),Onset String (HH:MM:SS),Offset String (HH:MM:SS),Seizure Type (FAS; FIAS; FBTC; Non-electrographic; Subclinical; Unknown),FileIndex,onset_datetime,offset_datetime,onset_datetime_buffer,offset_datetime_buffer,FileName
0,File,Epat02,File1,09:14:2017,13:05:11,17:05:11,NaN,0,2017-09-14 13:05:11,2017-09-14 17:05:11,2017-09-14 13:06:11,2017-09-14 17:04:11,Epat02_09142017_13051100.EDF
1,File,Epat02,File2,09:14:2017,17:05:22,21:05:22,NaN,1,2017-09-14 17:05:22,2017-09-14 21:05:22,2017-09-14 17:06:22,2017-09-14 21:04:22,Epat02_09142017_17052200.EDF
2,File,Epat02,File3,09:14:2017,21:05:34,01:05:34,NaN,2,2017-09-14 21:05:34,2017-09-15 01:05:34,2017-09-14 21:06:34,2017-09-15 01:04:34,Epat02_09142017_21053400.EDF
3,File,Epat02,File4,09:15:2017,01:05:45,05:05:45,NaN,3,2017-09-15 01:05:45,2017-09-15 05:05:45,2017-09-15 01:06:45,2017-09-15 05:04:45,Epat02_09152017_01054500.EDF
4,File,Epat02,File5,09:15:2017,05:05:56,09:05:56,NaN,4,2017-09-15 05:05:56,2017-09-15 09:05:56,2017-09-15 05:06:56,2017-09-15 09:04:56,Epat02_09152017_05055600.EDF


In [4]:
label_f = '~/Ephys/Data/all_pats_bipole_soz_labels.csv'
label_df = pd.read_csv(label_f, header=None)
label_df.columns = ['subj','bipole','label']



label_df = split_bipole(label_df)

label_df.label = label_df.label.apply(map_label)
label_df['bipole'] = label_df.bipole.apply(lambda x: x.replace(" ", ""))

# Calculating Connectivity Between Channels

It seems my task is much easier than I previously thought. There are designated recordings for my 4 periods of interested already cleaned and labelled by Derek and Graham. Bless.

Goal for this section's exploration: calculate ISH across 4 periods for 1 patient.
1. Find 4 recordings for patient
1. Calc Connectivity (PDC) for each recording
1. Im show some of those recordings
1. calculate inward-outward strengh for 4 periods - ISH
1. BONUS: Redo 1-4 with cross spectral coherence connectivity from the French

## Using Precomputed PDC Peri-Ictally

In [48]:
pat = 'Epat02'
conn_f = "/mnt/ernie_main/Ghassan/ephys/data/connectivity/Epat02/Epat02_10_FIAS_imcoh.mat"
conn_obj = load_mat(conn_f)

ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: datetime, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32)
ERROR:root:ERROR: MATLAB type not supported: table, (uint32)
ERROR:root:ERROR: MATLAB type not supported: string, (uint32

In [87]:
## loading the PDC and labels
regions = [reg[0] for reg in conn_obj['pdc']['seizure']['bip_labels_used']]
regions = format_bipoles(regions)


In [99]:
#get labels of each patient's bipoles as SOZ, PZ, etc
pat_df = label_df[label_df.subj==pat]
contact_designation = pat_df.label.values
bipoles = pat_df.bipole.values
bip_dict = dict(zip(bipoles, contact_designation))
pat_conn_labels = np.array([bip_dict[b] for b in regions])


In [100]:
def get_reg_inds(pat_conn_labels):
    soz_inds = np.where(pat_conn_labels == 'SOZ')[0]
    pz_inds = np.where(pat_conn_labels == 'PZ')[0]
    nz_bool = pat_conn_labels == 'NIZ'
    iz_bool = pat_conn_labels =='IZ'
    nz_inds = np.where(np.logical_or(nz_bool, iz_bool))[0]
    return soz_inds, pz_inds, nz_inds
soz_inds, pz_inds, nz_inds = get_reg_inds(pat_conn_labels)

In [97]:
soz_inds

array([35, 36])

In [66]:
BANDS = ['delta', 'theta', 'alpha', 'beta','gamma_l', 'gamma_H']
PERIOD = ['inter','pre','ictal','post']


In [86]:
def get_pdc_dict(conn_obj):
    inter_pdc = conn_obj['pdc']['seizure']['PDC_interictal']
    pre_pdc = conn_obj['pdc']['seizure']['PDC_pre']
    ictal_pdc = conn_obj['pdc']['seizure']['PDC_ictal']
    post_pdc = conn_obj['pdc']['seizure']['PDC_post']
    pdc_dict = dict(zip(PERIOD, [inter_pdc,pre_pdc,ictal_pdc,post_pdc]))
    return pdc_dict
pdc_dict = get_pdc_dict(conn_obj)

In [103]:
subj_id = pat
def assemble_net_conn(subj_id, pdc_dict,soz_inds,pz_inds,iz_inds):
    """Assemble net directed connectivity across periods of interest for all 
    frequency bands

    Args:
        subj_id (_type_): string of subject id
        pdc_dict (_type_): dictionary mapping period (str) to pdc (np.array)
        soz_inds (_type_): indices corresponding to soz nodes
        pz_inds (_type_): pz node indices
        iz_inds (_type_): iz node indices

    Returns:
        _type_: Dataframe of next connectivity for each region.
    """
    net_df = pd.DataFrame(columns=['subj', 'period', 'region', 'net_pdc', 'freq_band'])
    ind = 0 
    for period, pdc in pdc_dict.items():
        for b in range(6):
            band = BANDS[b]
            soz_in = pdc[b,:,soz_inds]
            soz_out = pdc[b,soz_inds,:]
            net_soz = np.nansum(soz_in) - np.nansum(soz_out)
            net_df.loc[ind] = [subj_id,period,'soz',net_soz,band]
            ind += 1

            pz_in = pdc[b,:,pz_inds]
            pz_out = pdc[b,pz_inds,:]
            net_pz = np.nansum(pz_in) - np.nansum(pz_out)
            net_df.loc[ind] = [subj_id,period,'pz',net_pz,band]
            ind +=1


            nz_in = pdc[b,:,nz_inds]
            nz_out = pdc[b,nz_inds,:]
            net_nz = np.nansum(nz_in) - np.nansum(nz_out)
            net_df.loc[ind] = [subj_id,period,'nz',net_nz,band]
            ind +=1
    return net_df

net_df = assemble_net_conn(subj_id,pdc_dict,soz_inds,pz_inds, iz_inds)

In [104]:
net_df

,subj,period,region,net_pdc,freq_band
0,Epat02,inter,soz,2.192529,delta
1,Epat02,inter,pz,-2.668998,delta
2,Epat02,inter,nz,0.476469,delta
3,Epat02,inter,soz,2.474896,theta
4,Epat02,inter,pz,-2.270837,theta
...,...,...,...,...,...
67,Epat02,post,pz,-7.668482,gamma_l
68,Epat02,post,nz,5.171305,gamma_l
69,Epat02,post,soz,2.652356,gamma_H
70,Epat02,post,pz,-8.198000,gamma_H
